In [1]:
using Pkg
Pkg.status(io=stdout)

     Project QXContexts v0.2.1
      Status `~/code/QXContexts/Project.toml`
  [1520ce14] AbstractTrees v0.3.4
  [c7e460c6] ArgParse v1.1.4
  [6e4b80f9] BenchmarkTools v1.0.0
  [052768ef] CUDA v3.3.0
  [864edb3b] DataStructures v0.18.9
  [5789e2e9] FileIO v1.9.1
  [033835bb] JLD2 v0.4.9
  [50d2b5c4] Lazy v0.15.1
  [da04e1cc] MPI v0.18.0
  [ebe7aa44] OMEinsum v0.4.4
  [9b87118b] PackageCompiler v1.2.6
  [189a3867] Reexport v1.1.0
  [98d24dd4] TestSetExtensions v2.0.0
  [a759f4b9] TimerOutputs v0.5.9
  [ddb6d928] YAML v0.4.7
  [ade2ca70] Dates
  [8f399da3] Libdl
  [37e2e46d] LinearAlgebra
  [56ddb016] Logging
  [44cfe95a] Pkg
  [9a3f8284] Random
  [cf7118a7] UUIDs


In [2]:
using QXContexts

┌ Info: Precompiling QXContexts [04c26001-d4a1-49d2-b090-1d469cf06784]
└ @ Base loading.jl:1317


OMEinsum: YOU FIND CUDA!


In [67]:
# prefix = "../../examples/ghz/ghz_5"
prefix = "../../examples/rqc/rqc_5_5_32"
cg, _ = parse_dsl_files([prefix * x for x in [".qx", ".jld2"]]...)

(ComputeGraph(ComputeNode{SaveCommand}: children: 1299, depth: 73, balance: 0.1506849315068493, max_degree: 23, Dict{Symbol, AbstractArray}(:data_5 => ComplexF64[1.0 + 0.0im, 0.7071067811865475 + 0.7071067811865475im], :data_1 => ComplexF64[1.0 + 0.0im, 0.0 + 0.0im], :data_6 => ComplexF64[0.4999999999999995 + 0.49999999999999983im 0.49999999999999967 - 0.49999999999999983im; 0.49999999999999967 - 0.49999999999999983im 0.49999999999999983 + 0.49999999999999983im], :data_7 => ComplexF64[0.4999999999999999 + 0.4999999999999999im -0.4999999999999999 - 0.4999999999999999im; 0.4999999999999999 + 0.4999999999999999im 0.4999999999999999 + 0.4999999999999999im], :data_3 => ComplexF64[-0.3446133714352872 + 0.0im 1.1381806476131544 + 0.0im; -1.1381806476131544 + 0.0im -0.3446133714352875 + 0.0im], :data_4 => ComplexF64[-1.0484937059720079 + 0.0im 0.5611368023131075 + 0.0im; 0.5611368023131075 + 0.0im 1.0484937059720079 + 0.0im], :data_2 => ComplexF64[0.7071067811865475 + 0.0im 0.7071067811865475 

In [68]:
cg

ComputeGraph(ComputeNode{SaveCommand}: children: 1299, depth: 73, balance: 0.1506849315068493, max_degree: 23, Dict{Symbol, AbstractArray}(:data_5 => ComplexF64[1.0 + 0.0im, 0.7071067811865475 + 0.7071067811865475im], :data_1 => ComplexF64[1.0 + 0.0im, 0.0 + 0.0im], :data_6 => ComplexF64[0.4999999999999995 + 0.49999999999999983im 0.49999999999999967 - 0.49999999999999983im; 0.49999999999999967 - 0.49999999999999983im 0.49999999999999983 + 0.49999999999999983im], :data_7 => ComplexF64[0.4999999999999999 + 0.4999999999999999im -0.4999999999999999 - 0.4999999999999999im; 0.4999999999999999 + 0.4999999999999999im 0.4999999999999999 + 0.4999999999999999im], :data_3 => ComplexF64[-0.3446133714352872 + 0.0im 1.1381806476131544 + 0.0im; -1.1381806476131544 + 0.0im -0.3446133714352875 + 0.0im], :data_4 => ComplexF64[-1.0484937059720079 + 0.0im 0.5611368023131075 + 0.0im; 0.5611368023131075 + 0.0im 1.0484937059720079 + 0.0im], :data_2 => ComplexF64[0.7071067811865475 + 0.0im 0.7071067811865475 +

In [69]:
memory_numbers(::AbstractCommand) = 0
memory_numbers(l::LoadCommand) = prod(l.dims)
memory_numbers(c::ContractCommand) = 1 << length(c.output_idxs) - (1 << length(c.left_idxs) + 1 << length(c.right_idxs))
memory_numbers(o::OutputCommand) = o.dim

update_set!(s, ::AbstractCommand) = nothing
update_set!(s, l::LoadCommand) = begin push!(s, l.name); nothing end
update_set!(s, v::ViewCommand) = begin setdiff!(s, [v.input_sym]); push!(s, v.output_sym); nothing end
update_set!(s, c::ContractCommand) = begin setdiff!(s, [c.left_name, c.right_name]); push!(s, c.output_name); nothing end
update_set!(s, c::SaveCommand) = begin setdiff!(s, [c.label]); push!(s, c.name); nothing end
update_set!(s, o::OutputCommand) = begin push!(s, o.name); nothing end

function track_memory(cmds)
    memory = 0
    syms = Set()
    history = Int[]
    for cmd in cmds
        cmd_memory = memory_numbers(cmd)
        update_set!(syms, cmd)
        memory += cmd_memory
        push!(history, memory)
    end
    memory, history
end

track_memory (generic function with 1 method)

In [70]:
m, h = track_memory(get_commands(cg))

(1, [2, 2, 6, 6, 8, 4, 2, 4, 4, 4  …  54, 52, 32, 30, 26, 16, 14, 8, 1, 1])

In [71]:
maximum(h) * 8 

67142896

In [66]:
using FileIO

In [42]:
d = load(prefix * ".jld2")

Dict{String, Any} with 4 entries:
  "data_2" => ComplexF64[0.707107+0.0im 0.707107+0.0im; 0.707107+0.0im -0.70710…
  "data_1" => ComplexF64[1.0+0.0im, 0.0+0.0im]
  "data_4" => ComplexF64[-1.18921+0.0im, 1.18921+0.0im]
  "data_3" => ComplexF64[-0.840896+0.0im 0.0+0.0im; 0.0+0.0im -0.840896+0.0im]…

In [43]:
d["data_4"]

2-element Vector{ComplexF64}:
 -1.189207115002721 + 0.0im
 1.1892071150027208 + 0.0im